In [ ]:
---
title: "Data Wrangling Report - Code Section"
author: "CINQUANTA Octave" 
date: "`r format(Sys.time(), '%b %Y')`"
output: 
  pdf_document: 
    toc: TRUE
always_allow_html: true
---

In [ ]:
knitr::opts_chunk$set(message=FALSE, echo = TRUE, tidy.opts=list(width.cutoff=60))

In [ ]:
library(reticulate)
library(Rcpp)

## Scraping

In [ ]:
%%python
# Importing libraries
from bs4 import BeautifulSoup
import requests
from lxml import etree
import re
import pandas as pd

The URL Scraper:

In [ ]:
%%python
# Setting the variables to be used in the scraper
next_page = ''
df = pd.DataFrame()
page_number = 2     # This variable determines the number of pages to be scraped

In [ ]:
%%python
# Looping through the pages to scrap
for k in range(page_number):
    # Collecting the page using the general URL and the suffix obtained below 
    page = requests.get('https://theses.fr/fr/?q='+re.sub("'","",next_page))    
    # Treating the page using Beautiful Soup and lxml
    soup = BeautifulSoup(page.text, 'html.parser')
    dom = etree.HTML(str(soup))
    # Setting the correct x-path and some useful variables
    X_path_url = '//*[@id="resultat"]/div[1]/div[1]/h2/a'
    j = 1
    links = []
    # Looping through the theses to scrap
    for i in range(10):
        # Using the x-path to select the thesis' link and store it in an array
        links.append('https://theses.fr'+dom.xpath(X_path_url)[0].get('href'))
        j = j+3
        # Changing the x-path to the one of the next thesis
        X_path_url = '//*[@id="resultat"]/div['+str(j)+']/div[1]/h2/a'
    # Obtaining the URL suffix of the next page and treating it with regex
    next_page_raw = dom.xpath('//*[@id="pagination"]/a[13]')[0].get('href')
    next_page = re.search('(\'(.*?)\')',next_page_raw).group(1)
    # Storing the obtained links in the data frame
    df = df.append(links)
    print('scraping page '+str(k+1)+'/'+str(page_number))

In [ ]:
%%python
# Exporting the links to a csv file
df.to_csv('url.csv')

The Author Scraper:

In [ ]:
%%python
# Setting up the new data frame
df2 = pd.DataFrame()

In [ ]:
%%python
for k in range(len(df)):
    # Collecting and treating the page, as well as setting up a variable
    page = requests.get(df.iloc[k,0])
    soup = BeautifulSoup(page.text, 'html.parser')
    dom = etree.HTML(str(soup))
    authors = []
    # There are two potential x-paths for the author
    # So the scraper has to try both the first one and use the second one if necessary
    try:
        authors.append(dom.xpath('//*[@id="ficheTitre"]/div[2]/h2/span')[0].text)
    except IndexError:
        authors.append(dom.xpath('//*[@id="ficheTitre"]/div[2]/h2/a/span')[0].text)
    # Storing the obtained author names in the new data frame
    df2 = df2.append(authors)
    print('scraping author '+str(k+1)+'/'+str(len(df)))

In [ ]:
%%python
# Removing the duplicate authors
df2 = df2.drop_duplicates()
# Exporting the authors to a csv file
df2.to_csv('authors.csv')

## Setting up the R environment

In [ ]:
# Setting the working directory
knitr::opts_knit$set(root.dir = "C:/Users/eleoc/Desktop/WORK/Y3S1")

In [ ]:
# Loading libraries
library(dplyr)
library(lubridate)
library(ggplot2)
library(hrbrthemes)
library(viridis)
library(naniar)
library(questionr)
library(missForest)
library(xtable)
library(stringr)
library(plotly)

In [ ]:
# Preparing a cleaner
rm(list=ls())

## Missing Data

In [ ]:
# Importing the data-set
theses <- read.csv('Data Wrangling\\theses_v2.csv',header=T, na.strings=c(""," ","NA"))

In [ ]:
# Plotting the missing data
vis_miss(theses,warn_large_data = FALSE) + 
  labs(title = "Missingness of the theses data")

In [ ]:
# Creating a table of the missing data
freq.na(theses)

In [ ]:
# Creating normally distributed array of number of pages, with 20% of missing values
pages <- matrix(rnorm(447644,200,50), 447644, 1) %>%
  prodNA(noNA = 0.2)
# Imputing the missing values with the column mean
pages[is.na(pages)] <- mean(pages,na.rm=TRUE)
# Rounding the values
pages <- round(pages)
# Inserting the column into the main data frame
theses$n.pages <- pages

## Common Issues

In [ ]:
# Formatting the defense dates
theses$Date.de.soutenance <- as.Date(theses$Date.de.soutenance,format='%d-%m-%Y')
# Creating two new columns corresponding to the day and month of defense
theses <- theses %>%
  mutate(Month = month(Date.de.soutenance)) %>%
  mutate(Day = day(Date.de.soutenance))

In [ ]:
# Counting the number of theses defended on the first of January
theses %>%
  filter(Month == 1 & Day == 1) %>%
  count(Month,Year) %>%
  colSums()

In [ ]:
# Obtaining the percentage of thesis defended on the first of January
(281012/447644)*100

In [ ]:
# Counting the number of theses not defended on the first of January per month
monthly_normal <- theses %>%
  filter(!(Month == 1 & Day == 1)) %>%
  count(Month,Year)
# Resuming this data per year
monthly_sum <- aggregate(monthly_normal$n,by=list(monthly_normal$Year), FUN=sum)

In [ ]:
# Counting the number of theses defended on the first of January per month
monthly11 <- theses %>%
  filter(Month == 1 & Day == 1) %>%
  count(Month,Year)

In [ ]:
# Renaming the Year column properly
names(monthly_sum)[names(monthly_sum) == 'Group.1'] <- 'Year'

In [ ]:
# Dropping the useless month column
monthly11 <- select(monthly11,2,3)

In [ ]:
# Merging the two data-sets to compare them
january <- merge(monthly_sum, monthly11, by = 'Year')

In [ ]:
# Computing the percentage of thesis defended on the first of January
january$percent <- (january$n/(january$n+january$x))*100

In [ ]:
# Plotting the data
ggplot(january) +
  geom_bar(aes(x=Year, y=percent), stat="identity", fill="skyblue", alpha=0.7) + 
  labs(title = "Evolution of the percentage of theses defended on the first of January")

In [ ]:
# Filtering out people having defended more than one thesis
unique_author_ID <- theses %>%
  filter(!duplicated(Identifiant.auteur))
# Obtaining the number of name homonyms
sum(duplicated(unique_author_ID$Auteur))

In [ ]:
# Computing the percentage of name homonyms
(9091/447644)*100

In [ ]:
# Looking for the Cecile Martin
theses %>%
  filter(Auteur == 'Cecile Martin' & Identifiant.auteur == '81323557')

In [ ]:
# Looking up the types of Identifiant.directeur
table(sapply(theses$Identifiant.directeur, class))

In [ ]:
# Looking up the number of director IDs containing commas
(sum(str_count(theses$Identifiant.directeur,",") != 0))/2

In [ ]:
# Computing the percentage of director IDs containing commas
(29554/447644)*100

## Outliers

In [ ]:
# Creating a data frame with the frequency of directors
directors <- as.data.frame(table(theses$Directeur.de.these))
# Renaming its columns
directors <- directors %>%
  rename(Name = Var1, n = Freq)

In [ ]:
# Looking at the directors that directed a lot of theses
directors %>%
  filter(n >= 100)

In [ ]:
# Creating a data frame with the frequency of authors
authors <- as.data.frame(table(theses$Auteur))
# Renaming its columns
authors <- authors %>%
  rename(Name = Var1, n = Freq)

In [ ]:
# Looking at the authors that defended a lot of theses
authors %>%
  filter(n >= 10)

In [ ]:
# Looking for early career scientists
theses %>%
  filter(Auteur == 'Philippe Michel' & Identifiant.auteur == '83251103')

## Preliminary Results

In [ ]:
# Renaming the languages column
theses <- theses %>%
  rename(langage = Langue.de.la.these)

In [ ]:
# Counting the languages
langages <- theses %>%
  count(langage,Year) %>%
  mutate(langage = case_when(langage == 'fr' ~ 'french', langage == 'en' ~ 'english',
                             langage == 'fren' | langage == 'enfr' ~ 'bilingual',
                             TRUE ~ 'others')) %>%
  group_by(langage,Year) %>%
  summarise(n = sum(n))

In [ ]:
# Removing the useless years
langages <- langages %>%
  filter(Year >= 1988 & Year <= 2018)

In [ ]:
# Plotting the data using ggplot
ggplot(langages, aes(x=Year, y=n, 
                     fill=factor(langage, 
                                 levels = c('french','bilingual','others','english')))) + 
  geom_area(position = position_stack(reverse = TRUE)) +
  labs(fill='Language') +
  scale_fill_viridis_d() + 
  labs(title = "Evolution of the number of theses defended per language")

In [ ]:
# Creating a variable per language 
lang_bilingual <- subset(langages,langage == 'bilingual')
lang_french <- subset(langages,langage == 'french')
lang_english <- subset(langages,langage == 'english')
lang_others <- subset(langages,langage == 'others')

In [ ]:
# Creating a new language variable for use of plotly
langages2 <- lang_bilingual[2]
langages2 <- langages2 %>%
  mutate(bilingual = lang_bilingual$n,french = lang_french$n,
         english = lang_english$n,others = lang_others$n)

In [ ]:
# Plotting the data using plotly
fig <- plot_ly(langages2, x = ~Year, y = ~bilingual, 
               name = 'Bilingual', type = 'scatter', 
               mode = 'none', stackgroup = 'one', fillcolor = '#F5FF8D')
fig <- fig %>% add_trace(y = ~others, name = 'Others', fillcolor = '#700961')
fig <- fig %>% add_trace(y = ~english, name = 'English', fillcolor = '#50CB86')
fig <- fig %>% add_trace(y = ~french, name = 'French', fillcolor = '#4C74C9')
fig <- fig %>% layout(title = 'Evolution of the number of theses defended per language',
         xaxis = list(title = "",
                      showgrid = FALSE),
         yaxis = list(title = "Language",
                      showgrid = FALSE))
fig

In [ ]:
# Obtaining the monthly amount of theses
monthly <- theses %>%
  filter(!(Month == 1 & Day == 1)) %>%
  count(Month,Year)

In [ ]:
# Removing the useless years
monthly <- monthly %>%
  filter(Year >= 2013 & Year != 2020)

In [ ]:
# Computing the yearly percentages
monthly <- monthly %>%
  group_by(Year) %>% 
  mutate(percent = n/sum(n)*100)

In [ ]:
# Obtaining the means and standard deviations of those percentages
monthly <- monthly %>%
  group_by(Month) %>%
  mutate(mean = mean(percent)) %>%
  mutate(sd = sd(percent))

In [ ]:
# Collecting the data in a simpler matrix
superior_monthly <- unique(select(monthly,1,5,6))

In [ ]:
# Plotting the data
ggplot(superior_monthly) +
  geom_bar(aes(x=Month, y=mean), stat="identity", fill="skyblue", alpha=0.7) +
  geom_errorbar(aes(x=Month, ymin=mean-sd, ymax=mean+sd), 
                width=0.4, colour="orange", alpha=0.9, size=1.3) + 
  labs(title = "Repartition of the percentage of theses defended per month")